In [31]:
import os
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from time import time
import cv2
from unet import UNet
import PIL
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import pydicom
import re
from sklearn.cluster import KMeans
from skimage import morphology
from skimage import measure
from skimage import exposure
import glob
import torchvision
from PIL import Image
from statistics import mean
from torchvision.utils import make_grid
from torchvision import datasets, models, transforms
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, accuracy_score, classification_report
import seaborn as sn  # for heatmaps
import time
import copy
import augmentations as a
import helper_functions as hlp


In [32]:
no_brands = 3

# Reading train, validation and test sets
train_set = pd.read_csv('/home/judy/devbox/cspine_hardware/CSC2541 - University of Toronto/Jahangir/Train_DCM_Labels_Strat.csv')
val_set = pd.read_csv('/home/judy/devbox/cspine_hardware/CSC2541 - University of Toronto/Jahangir/Val_DCM_Labels_Strat.csv')
test_set = pd.read_csv('/home/judy/devbox/cspine_hardware/CSC2541 - University of Toronto/Jahangir/Test_DCM_Labels_Strat.csv')
annotations_df = pd.read_csv('/home/judy/devbox/cspine_hardware/CSC2541 - University of Toronto/Jahangir/annotations.csv')
# Data path
dcom_data_path = '/home/judy/devbox/cspine_hardware'
#Reference Image: 
ref_image_1 = pydicom.dcmread('Obsolete/IM-2275-0001.dcm')


### Anterior and Posterior seperation

In [33]:
anterior_df = pd.read_csv(dcom_data_path + 
                          '/CSC2541 - University of Toronto/Master_Anterior_HDW.csv', header = 'infer')
posterior_df = pd.read_csv(dcom_data_path + 
                          '/CSC2541 - University of Toronto/Master_Posterior_HDW.csv', header = 'infer')


### Create training, testing and validation data sets for Anterior and Posterior implants

In [34]:
hlp.split_data2(anterior_df, 'anterior_proc_data')
hlp.split_data2(posterior_df, 'posterior_proc_data')

In [35]:
train_set = pd.read_csv('Train_anterior_data.csv')
val_set = pd.read_csv('Val_anterior_data.csv')
test_set = pd.read_csv('Test_anterior_data.csv')

### Specify specific inputs depending on run:


In [36]:
# Update where images will be stored:
home_folder = '/home/judy/devbox/cspine_hardware/CSC2541 - University of Toronto/Jahangir/data_anterior_3/' 

# Apply Manual Image cropping:
crop_ind = 1 # Set 0 if no
process_ind = 0 # Set 0 if no

# LB and UB to crop Image rows - must be between 0 and 1: 
LB_row = 0.35
UB_row = 0.70
#LB and UB to crop Image columns - must be between 0 and 1:
LB_col = 0.40
UB_col = 0.70
# Clipping Value for Adaptive Histogram Equalization - Clipping Limit between 0 and 1: 
clip_val = 0.1
# Percentile bounds for rescaling intensities - between 0 and 100: 
low_p = 70
high_p = 95


### Create training, validation and testing folders and subsequent sub brand subfolders:

In [37]:
# Home data folders
train_subfolder = home_folder + 'train/'                                                            # Training sub folder
test_subfolder = home_folder + 'test/'                                                              # Testing sub folder
val_subfolder = home_folder + 'val/'                                                                # Validation sub folder

#Create folders for home folder, train, validation and test
hlp.create_folders(home_folder)
hlp.create_folders(train_subfolder)
hlp.create_folders(val_subfolder)
hlp.create_folders(test_subfolder)

#List of sub fodlers
sub_folder_list = [train_subfolder, val_subfolder, test_subfolder]

# Create brand sub folders
for i in sub_folder_list:
    for j in range(no_brands):
        hlp.create_folders(i+str(j+1)+'/')
print('Finished!')

Finished!


### Copy images to folder:

Copy DICOM images to PNG images after performing photometric interpretation (Monochromatic Interpretation), followed by cropping (if specified) and contrast stretching (if specified)

In [38]:
set_df_list = [train_set, val_set, test_set]
hlp.to_png_3(no_brands, dcom_data_path,sub_folder_list, crop_ind, process_ind, set_df_list,
            ref_image_1, LB_row, UB_row, LB_col, UB_col, clip_val, low_p, high_p)

Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
Finished
F